In [2]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from datetime import datetime, date, timedelta
import os
import xarray as xr

In [17]:
df = pd.read_csv("../data/processed/predictors/formation.csv", parse_dates=["formation_datetime"])

In [22]:
# time as timestamp
#
with xr.open_dataset(os.path.join("../data", "raw", "air", "air.2024.nc")) as ds:
	print(ds.variables)

Frozen({'time_bnds': <xarray.Variable (time: 366, nbnds: 2)> Size: 6kB
[732 values with dtype=datetime64[ns]]
Attributes:
    long_name:  Time Boundaries, 'air': <xarray.Variable (time: 366, level: 17, lat: 73, lon: 144)> Size: 262MB
[65405664 values with dtype=float32]
Attributes: (12/13)
    long_name:      daily mean 6-hourly Air Temperature on Pressure Levels
    units:          degK
    precision:      2
    GRIB_id:        11
    GRIB_name:      TMP
    var_desc:       Air temperature
    ...             ...
    level_desc:     Pressure Levels
    statistic:      Mean
    parent_stat:    Individual Obs
    standard_name:  air_temperature
    valid_range:    [137.5 362.5]
    actual_range:   [186.02501 315.95   ], 'level': <xarray.IndexVariable 'level' (level: 17)> Size: 68B
array([1000.,  925.,  850.,  700.,  600.,  500.,  400.,  300.,  250.,  200.,
        150.,  100.,   70.,   50.,   30.,   20.,   10.], dtype=float32)
Attributes:
    units:               millibar
    actual_ran

In [19]:
file_path = "air.{year}.nc"
def get_air_by_date_offset(ts, lat, lon, level, days_earlier):
	form_date = ts.date() # Convert from Timestamp to datetime.date
	target_date = form_date - timedelta(days = days_earlier)
	# target_prev_sunday = target_date - timedelta(days = (target_date.weekday() + 1) % 7)
	# target_first_of_month = date(target_date.year, target_date.month, 1)

	if form_date.year >= 2024:
		f = form_date.year
	elif form_date.year >= 1979:
		f = "1979-2023"
	else:
		f = "1851-1978"

	path = file_path.format(year=f)

	with xr.open_dataset(os.path.join("../data", "raw", "air", path)) as ds:
		air_temp = ds['air'].sel(
			lat = lat,
			lon = lon,
			level = level,
			time = target_date,
			method = 'nearest'
		).item()

	return air_temp


In [23]:
lead_times = [0]
levels = [250, 500]
for lead in lead_times:
	for level in levels:
		df[f'air_temp_{level}mb'] = df.apply(axis='columns', func = lambda row: get_air_by_date_offset(row['formation_datetime'], row['formation_lat'], row['formation_lon'], level, lead))
df.head()

,code,formation_datetime,formation_lat,formation_lon,air_temp_250mb,air_temp_500mb
0,AL011851,1851-06-25 00:00:00,28.0,265.2,236.364075,273.410400
1,AL021851,1851-07-05 12:00:00,22.2,262.4,237.293304,270.061523
2,AL031851,1851-07-10 12:00:00,12.0,300.0,232.705948,267.771210
3,AL041851,1851-08-16 00:00:00,13.4,312.0,231.948059,267.918762
4,AL051851,1851-09-13 00:00:00,32.5,286.5,229.951294,265.977814


In [24]:
results = df.drop(axis='columns', labels=['formation_lat','formation_lon','formation_datetime'])
results.head()

,code,air_temp_250mb,air_temp_500mb
0,AL011851,236.364075,273.410400
1,AL021851,237.293304,270.061523
2,AL031851,232.705948,267.771210
3,AL041851,231.948059,267.918762
4,AL051851,229.951294,265.977814


In [25]:
results.to_csv('../data/processed/air.csv', index=False)